In [37]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import pandas as pd

In [38]:
# Define file and directory names
file_name = '0_61938.csv'
data_dir = 'ohlc_data'
parent_dir = 'forexPredictor'
repo = 'Repos_git'
repo_dir = 'Make_Money_with_Tensorflow_2.0'
# Get the absolute base directory dynamically
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move up one level

# Construct the full file path in an OS-independent way
data_path = os.path.join(base_dir, repo, repo_dir, parent_dir, data_dir, file_name)

# Check if the file exists before using it
if os.path.exists(data_path):
    print(f"✅ File found: {data_path}")
else:
    print(f"❌ Error: File not found at {data_path}")


ucols=['Open', 'High', 'Low', 'Close']
data = pd.read_csv(data_path, usecols=ucols)
data.reset_index(drop=True, inplace=True)
data.head()


✅ File found: D:\Repos_git\Make_Money_with_Tensorflow_2.0\forexPredictor\ohlc_data\0_61938.csv


,Open,High,Low,Close
0,1.20997,1.21089,1.20966,1.20999
1,1.20481,1.20569,1.20479,1.20538
2,1.20537,1.20574,1.20341,1.20553
3,1.20556,1.20689,1.20442,1.20469
4,1.20468,1.20599,1.20380,1.20573


In [ ]:
ohlc_data = data

# Normalize the data
scaler = MinMaxScaler()
ohlc_scaled = scaler.fit_transform(ohlc_data)

# Convert OHLC data into sequences (LSTM requires 3D input)
sequence_length = 6  # Number of time steps to consider
X = []
for i in range(len(ohlc_scaled) - sequence_length):
    X.append(ohlc_scaled[i:i+sequence_length])

X = np.array(X)  # Shape: (num_samples, sequence_length, num_features)

# Define LSTM encoder
def create_encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = LSTM(128, return_sequences=True)(inputs)
    x = LSTM(64)(x)
    x = Dense(32, activation='relu')(x)
    return Model(inputs, x)

# Create encoder model
encoder = create_encoder((sequence_length, 4))

# Extract feature embeddings
features = encoder.predict(X)

# Apply K-Means clustering
num_clusters = 3  # Buy (2), Sell (1), Hold (0)
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(features)  # Assign cluster labels

# Convert cluster labels to categorical for classification training
clusters_one_hot = to_categorical(clusters, num_classes=num_clusters)

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, clusters_one_hot, test_size=0.2, random_state=42)

# Define classification model
classifier_output = Dense(num_clusters, activation="softmax")(encoder.output)
final_model = Model(encoder.input, classifier_output)

# Compile and train classifier
final_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
final_model.fit(X_train, y_train, epochs=20, batch_size=1, validation_data=(X_test, y_test))

# Evaluate model on test data
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# --- Prediction on New Data ---
new_data = np.array([
    [1.20800, 1.20950, 1.20760, 1.20910],  # Example OHLC row
    [1.20920, 1.21080, 1.20890, 1.21010],
    [1.21000, 1.21150, 1.20980, 1.21090]
])

# Normalize and reshape new data
new_data_scaled = scaler.transform(new_data)
new_data_sequence = np.array([new_data_scaled])  # Shape (1, sequence_length, features)

# Make predictions
predictions = final_model.predict(new_data_sequence)
predicted_class = np.argmax(predictions, axis=1)[0]  # Get cluster label (0, 1, or 2)

# Convert prediction to label
labels = {0: "Hold", 1: "Sell", 2: "Buy"}
print(f"Predicted Action: {labels[predicted_class]}")


1936/1936 [==============================] - 5s 2ms/step


C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/20
49545/49545 [==============================] - 167s 3ms/step - loss: 0.1067 - accuracy: 0.9572 - val_loss: 0.0428 - val_accuracy: 0.9785
Epoch 2/20
10404/49545 [=====>........................] - ETA: 2:04 - loss: 0.0656 - accuracy: 0.9733